# Get OpenStreetMap Data

For this, we use the OSMPythonTools package as recommended by the OSM wiki. It allows us to access the Overpass API (a read-only API, perfect for what we are set to do), the OSM API, and the Nominatim access point. More info here: https://wiki.openstreetmap.org/wiki/OSMPythonTools

#### Which object does the way with the id 5887599 represent?

In [3]:
from OSMPythonTools.api import Api
api = Api()
way = api.query('way/5887599')

#### How many trees are in the OSM data of Vienna? And how many trees have there been in 2013?

In [7]:
# Find the id for the area Vienna
from OSMPythonTools.nominatim import Nominatim
nominatim = Nominatim()
areaId = nominatim.query('Vienna, Austria').areaId()
print('Vienna Id:', areaId)

Vienna Id: 3600109166


In [9]:
# query the Overpass API for the numbe of trees in Vienna using the id we just retrieved.
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()
query = overpassQueryBuilder(area=areaId, elementType='node', selector='"natural"="tree"', out='count')
result = overpass.query(query)
print('there are', result.countElements(), 'trees in Vienna')

there are 143275 trees in Vienna


In [10]:
# How many were there in 2013 ?
result = overpass.query(query, date='2013-01-01T00:00:00Z', timeout=60)
print('there were', result.countElements(), 'trees in Vienna in 2013')

[overpass] downloading data: [timeout:60][date:"2013-01-01T00:00:00Z"][out:json];area(3600109166)->.searchArea;(node["natural"="tree"](area.searchArea);); out count;


there were 127690 trees in Vienna in 2013


#### How did the number of trees changed in Vienna, Paris and Berlin changed since 2013? 

In [11]:
from collections import OrderedDict
from OSMPythonTools.data import Data, dictRangeYears, ALL
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

# The question has two "dimensions": the dimension of time, and the dimension of different cities: 
dimensions = OrderedDict([
    ('year', dictRangeYears(2013, 2017.5, 1)),
    ('city', OrderedDict({
        'berlin': 'Berlin, Germany',
        'paris': 'Paris, France',
        'vienna': 'Vienna, Austria',
    })),
])

In [ ]:
overpass = Overpass()
def fetch(year, city):
    areaId = nominatim.query(city).areaId()
    query = overpassQueryBuilder(area=areaId, elementType='node', selector='"natural"="tree"', out='count')
    return overpass.query(query, date=year, timeout=60).countElements()
data = Data(fetch, dimensions)

[nominatim] downloading data: search
[overpass] downloading data: [timeout:60][date:"2013-01-01T00:00:00Z"][out:json];area(3600062422)->.searchArea;(node["natural"="tree"](area.searchArea);); out count;
[nominatim] downloading data: search
[overpass] downloading data: [timeout:60][date:"2013-01-01T00:00:00Z"][out:json];area(3600007444)->.searchArea;(node["natural"="tree"](area.searchArea);); out count;


In [ ]:
# Plot it in a .jpg picture
data.plot(city=ALL, filename='example.png')

In [ ]:
# Or get it as a .csv
data.select(city=ALL).getCSV()